In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Step 1: Generate a random matrix with 100 rows and 1000 columns
np.random.seed(42)  # For reproducibility
tfidf_matrix = np.random.rand(100, 1000)

# Step 2: Apply dimensionality reduction using PCA or t-SNE
# Option 1: Using PCA
pca = PCA(n_components=2)
reduced_matrix_pca = pca.fit_transform(tfidf_matrix)

# Option 2: Using t-SNE (uncomment if you prefer t-SNE over PCA)
tsne = TSNE(n_components=2, random_state=42)
reduced_matrix_tsne = tsne.fit_transform(tfidf_matrix)

# Step 3: Plot the reduced matrix with labels
plt.figure(figsize=(10, 8))
plt.scatter(reduced_matrix_pca[:, 0], reduced_matrix_pca[:, 1], c='blue', label='Documents')
# plt.scatter(reduced_matrix_tsne[:, 0], reduced_matrix_tsne[:, 1], c='blue', label='Documents')  # Uncomment if using t-SNE

# Label each point with its document index
for i in range(reduced_matrix_pca.shape[0]):
    plt.text(reduced_matrix_pca[i, 0] + 0.02, reduced_matrix_pca[i, 1] + 0.02, str(i), fontsize=9)
    # plt.text(reduced_matrix_tsne[i, 0] + 0.02, reduced_matrix_tsne[i, 1] + 0.02, str(i), fontsize=9)  # Uncomment if using t-SNE

plt.title('2D Visualization of TF-IDF Vectors')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import pairwise_distances

# Compute the centroid (mean vector) of the TF-IDF matrix
centroid = np.mean(tfidf_matrix, axis=0)

# Compute the Euclidean distance of each document from the centroid
distances = np.linalg.norm(tfidf_matrix - centroid, axis=1)

# Threshold based on mean and standard deviation
mean_distance = np.mean(distances)
std_distance = np.std(distances)

# Consider documents with distances > mean + 2*std as outliers
outlier_indices = np.where(distances > mean_distance + 2 * std_distance)[0]

print(f"Outlier document indices: {outlier_indices}")


In [ ]:
from sklearn.cluster import DBSCAN

# Apply DBSCAN to find outliers
dbscan = DBSCAN(eps=5, min_samples=5, metric='euclidean')
labels = dbscan.fit_predict(tfidf_matrix)

# Outliers are labeled as -1
outlier_indices = np.where(labels == -1)[0]

print(f"Outlier document indices: {outlier_indices}")


In [ ]:
from sklearn.ensemble import IsolationForest

# Fit the Isolation Forest model
iso_forest = IsolationForest(contamination=0.05, random_state=42)
outlier_labels = iso_forest.fit_predict(tfidf_matrix)

# Outliers are labeled as -1
outlier_indices = np.where(outlier_labels == -1)[0]

print(f"Outlier document indices: {outlier_indices}")


In [ ]:
from sklearn.decomposition import PCA

# Reduce dimensionality with PCA
pca = PCA(n_components=2)
reduced_matrix = pca.fit_transform(tfidf_matrix)

# Compute distances in the reduced space
centroid_reduced = np.mean(reduced_matrix, axis=0)
distances_reduced = np.linalg.norm(reduced_matrix - centroid_reduced, axis=1)

# Consider outliers in the reduced space
mean_distance_reduced = np.mean(distances_reduced)
std_distance_reduced = np.std(distances_reduced)

outlier_indices = np.where(distances_reduced > mean_distance_reduced + 2 * std_distance_reduced)[0]

print(f"Outlier document indices: {outlier_indices}")


In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# Assuming tfidf_matrix is already defined
k = 2  # Number of clusters

kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(tfidf_matrix)

labels = kmeans.labels_

# Visualize the clusters using PCA to reduce dimensions to 2
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduced_data = pca.fit_transform(tfidf_matrix)

plt.figure(figsize=(10, 8))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='viridis')
plt.title("K-Means Clustering of TF-IDF Vectors")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(label="Cluster")
plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

# Perform hierarchical clustering
hier_clustering = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
labels = hier_clustering.fit_predict(tfidf_matrix)

# Visualize the dendrogram
linked = linkage(tfidf_matrix, 'ward')

plt.figure(figsize=(10, 7))
dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=True)
plt.title("Hierarchical Clustering Dendrogram")
plt.show()

# Plot clusters
reduced_data = PCA(n_components=2).fit_transform(tfidf_matrix)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='plasma')
plt.title("Hierarchical Clustering of TF-IDF Vectors")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(label="Cluster")
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN

# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='cosine')
labels = dbscan.fit_predict(tfidf_matrix)

# Plot clusters
reduced_data = PCA(n_components=2).fit_transform(tfidf_matrix)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='rainbow')
plt.title("DBSCAN Clustering of TF-IDF Vectors")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(label="Cluster")
plt.show()


In [ ]:
from sklearn.cluster import SpectralClustering

# Perform spectral clustering
spectral = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', random_state=42)
labels = spectral.fit_predict(tfidf_matrix)

# Plot clusters
reduced_data = PCA(n_components=2).fit_transform(tfidf_matrix)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='coolwarm')
plt.title("Spectral Clustering of TF-IDF Vectors")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(label="Cluster")
plt.show()


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Perform LDA to identify topics
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(tfidf_matrix)

# Get the topic distribution for each document
topic_distribution = lda.transform(tfidf_matrix)

# Assign each document to the most probable topic
labels = np.argmax(topic_distribution, axis=1)

# Plot clusters based on topics
reduced_data = PCA(n_components=2).fit_transform(tfidf_matrix)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='autumn')
plt.title("LDA-Based Clustering of TF-IDF Vectors")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(label="Topic")
plt.show()


In [ ]:
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import numpy as np

# Assuming tfidf_matrix is already defined

# Step 1: Apply Truncated SVD (LSA) to reduce dimensions
n_components = 2  # Reduce to 2 dimensions for visualization
lsa = TruncatedSVD(n_components=n_components, random_state=42)
lsa_result = lsa.fit_transform(tfidf_matrix)

# Step 2: Plot the documents in the reduced 2D space
plt.figure(figsize=(10, 8))
plt.scatter(lsa_result[:, 0], lsa_result[:, 1], c='red', edgecolor='k', s=50)
plt.title("LSA (Latent Semantic Analysis) of TF-IDF Vectors")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.grid(True)
plt.show()

# Optional: Print the explained variance ratio to understand how much variance is captured
explained_variance = lsa.explained_variance_ratio_
print(f"Explained variance by the first 2 components: {np.sum(explained_variance[:2])}")
